In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [5]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [14]:
garbageCollectionDate[16]

'Whittlesea-garbage-collection-zones.geojson'

In [15]:
readInJson(garbageCollectionDate[16]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_59083f37_5f22_48bf_8210_a0d3a26d7084.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Wednesday Week 1', 'desc_': 'Week 1'..."
1,Feature,ckan_59083f37_5f22_48bf_8210_a0d3a26d7084.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Wednesday Week 2', 'desc_': 'Week 2'..."


In [11]:
df = readInJson(garbageCollectionDate[16])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [7]:
df.iloc[0,0]

{'name': 'Wednesday Week 1',
 'desc_': 'Week 1',
 'rub_day': 'Wednesday',
 'rub_weeks': '1',
 'rub_start': '2017-01-04',
 'rub_scope': 'all',
 'rub_desc': None,
 'rec_day': 'Wednesday',
 'rec_weeks': '2',
 'rec_start': '2017-01-04',
 'rec_scope': 'all',
 'rec_desc': None,
 'grn_day': 'Wednesday',
 'grn_weeks': '2',
 'grn_start': '2017-01-11',
 'grn_scope': 'optional',
 'grn_desc': None,
 'hw_scope': 'booked',
 'hw_desc': None,
 'info_url': 'http://www.whittlesea.vic.gov.au/waste-environment/bins-and-waste-collection/'}

In [12]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''
        
    returningStr = rub_colFreq + rub_colDay
    
    if len(returningStr.split())>1:
         return returningStr
    else: 
        return 'Unavailable'

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

df.drop('properties', axis = 1, inplace = True)

In [13]:
whittlesea = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

whittlesea = whittlesea.loc[whittlesea.collection_time!= 'Unavailable']

whittlesea.to_csv('../GBD_whittleseaa.csv', index = False)